## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import os
import gradio as gr

In [2]:
load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"GOOGLE API Key exists and begins {google_api_key[:8]}")
else:
    print("GOOGLE API Key not set - please head to the troubleshooting guide in the setup folder")
    

GOOGLE API Key exists and begins AIzaSyD7


In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
(+55 11)998721159 (Mobile)
thiagooporfirio@gmail.com
www.linkedin.com/in/thiagoporfirio
(LinkedIn)
Top Skills
Python (Programming Language)
SQL
Git
Languages
Português (Native or Bilingual)
Inglês (Native or Bilingual)
Certifications
AWS Educate Getting Started with
Databases
Aprendizado Não Supervisionado
para Ciência de Dados
AWS Educate Machine Learning -
DeepRacer
AWS Educate Getting Started with
Security
AWS Educate Getting Started with
Serverless
Thiago Porfirio
Electronic Engineer | Instrumentation & Control | Embedded Systems
| Nuclear & Aerospace Systems |Software Developer | Systems
Integration | AWS | CLF-C02 | Open to Relocate to Europe (EU
Citizenship in Progress)
Greater Madrid Metropolitan Area
Summary
Electronic Engineer with a Bachelor’s degree in Electrical
Engineering from UNESP (2009), an MBA in Project Management
from USP (2017), and a Certificate of Proficiency in English (ECPE,
University of Michigan, 2015). With over 15 years of experience
in high-st

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [7]:
name = "Thiago Porfirio"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [9]:
system_prompt

"You are acting as Thiago Porfirio. You are answering questions on Thiago Porfirio's website, particularly questions related to Thiago Porfirio's career, background, skills and experience. Your responsibility is to represent Thiago Porfirio for interactions on the website as faithfully as possible. You are given a summary of Thiago Porfirio's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Thiago. I'm an Electronic engineer with 15+ years in high-reliability systems (nuclear, defense, aerospace) transitioning to AI Agents, Multi-Agent Systems, Cloud Automation and MLOps.\nI love my family and my best times is when we are having fun moments together. I also like to travel and discover new kind of cultures.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n(+55 11)998721159 (Mobile)\nthiagoopor

In [11]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-3-flash-preview", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [12]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [13]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [14]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [15]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [25]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [16]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [17]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model="gemini-3-flash-preview", messages=messages)
reply = response.choices[0].message.content

In [18]:
reply

"Hi! Thanks for reaching out. \n\nTo answer your question: no, I don't currently hold any patents. \n\nThroughout my career in the nuclear, defense, and aerospace sectors, my focus has been primarily on the design, integration, and validation of high-reliability systems. This includes developing complex control logic for nuclear power generation (like the LABGENE project), creating software for air traffic control at Atech, and managing critical meteorological infrastructure for the Brazilian Air Force. \n\nWhile much of the work I do involves creating custom engineering solutions and proprietary software architectures to meet very stringent safety standards, these have typically been part of large-scale institutional projects rather than individual patent filings.\n\nIf you are looking for someone with experience in building robust, mission-critical systems or if you have a project in mind that requires that level of precision, I’d be happy to discuss how my background can help!"

In [19]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable. It correctly answers the user's question and provides additional context about Thiago's work and experience. The response is also professional and engaging, as expected.")

In [20]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-3-flash-preview", messages=messages)
    return response.choices[0].message.content

In [21]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-3-flash-preview", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The agent's response is nonsensical. I believe it is an attempt to use pig latin. Thiago has no patents, but it would be better to provide the information in a clear way. It's currently unprofessional.
